## Extract Goal Graph from Python Code

This notebook extracts the goal graph from the generated python code.

In [1]:
import json

data_path = 'data1'

results = json.load(open('%s/models.json' % data_path, 'r'))

In [2]:
import networkx as nx

# convert local symbol table to goal graph
def get_graph(l):
    # reverse lookup from object to variable name
    rev = {o.text:v for v, o in l.items() if type(o).__name__ == 'Goal'}


    # variable name pairs in refined_by relation
    missing = []
    edges = []
    for v1, o1 in l.items():
        if type(o1).__name__ != 'Goal':
            continue
        for o2 in o1.is_refined_by: #is_why_we_satisfy_the_goal
            if o2.text in rev:
                edges.append((v1, rev[o2.text]))
            else:
                print('Missing obj for: %s' % o2.text)
                missing.append(o2.text)
    
    return edges, {v:t for t,v in rev.items()}

def get_implied_goals(l):
    if not 'implied_goals' in l:
        return []
        
    implied = [v for v, o in l.items() if type(o).__name__ == 'Goal' and o in l['implied_goals']]
    return implied

def get_cycles(edges):
    g = nx.DiGraph(edges)
    return [c for c in nx.simple_cycles(g)]

In [3]:
graphs = {}
for i in results.keys():
    graphs[i] = []
    
    for j in range(len(results[i])):
        # retrieve program and compute goal graph
        p, r, program = results[i][j]['pass']
        local = {}
        exec('\n'.join(program), {}, local)
        edges, nodes = get_graph(local)

        # remove implied edges, which yield false positives
        implied = get_implied_goals(local)
        graphs[i].append([nodes, edges, implied])

In [4]:
json.dump(graphs, open('%s/graphs.json' % data_path, 'w+'))